In [1]:
import sys
sys.path.append("R:/01_Analytics/plutusIO/02_code/Python")
from context.instruments import CurrentAccount, Mortgage
from context.assets import RealEstate
from context.agreements import RentalAgreement, UtilitiesAgreement, EmployeeContract
from context.party import Party
from core.plots import PlutusPlotter

In [2]:
# --- Portfolio ---
# Current Account
current_account = CurrentAccount(id='001'
                                , name='Bezny ucet Rado'
                                , current_outstanding=10000
                                , monthly_cost= 100
                                , cnit=0.0)

# Mortgage
mortgage = Mortgage(id='002'
                    , name='hypoteka csob'
                    , principal=3e6
                    , cnit=0.02
                    , maturity_in_years=30)

# Flat
flat = RealEstate(id='003'
                    , name='Byt'
                    , current_market_value=5e6
                    , property_tax = 2000
                    , renovation_costs = 2e5
                    , renovation_steps = {5, 10}
                    , house_community_costs = 6000)


# EmployeeContract
employeeContract = EmployeeContract(id='003'
                                    , name='zamestnanec Credo'
                                    , average_yearly_validity = 1.0
                                    , salary = 65000
                                    , bonus_steps= set(range(6,361,6))
                                    , bonus_rate = 0.3
                                    , yearly_salary_trend = 0.05
                                    , income_tax_rate = 0.42)

# --- Party ---
party = Party(name= 'Rado'
             , initial_free_resources = 1e5
             , initial_portfolio = {current_account, mortgage, flat, employeeContract}
             , monthly_expenditures = 20000)

In [3]:
for step in range(0, 36):
    party.live(step)

KeyError: 0

In [ ]:
plotter = PlutusPlotter(party)
plotter.plot_resources_evolution()

In [ ]:
mortgage.calculateNumberOfPeriods()
mortgage.calculateMonthlyPayment()
paid_principal = 0
paid_interest = 0

print(mortgage.monthly_interest_rate)

for step in range(0,mortgage.number_of_periods+1):
    mortgage.calculateInterestPayment(step)
    mortgage.calculatePrincipalPayment(step)
    mortgage.calculateOutstandingAmount(step)
    
    paid_principal = paid_principal + mortgage.principal_payment[step]
    paid_interest = paid_interest + mortgage.interest_payment[step]
    
    print('[step ' + str(step) + '] outstanding: ' + "%.2f"%mortgage.outstanding_amount[step] +
         ' | mp: ' + "%.2f"%mortgage.monthly_payment +
         ' | pp: ' + "%.2f"%mortgage.principal_payment[step] +
         ' | ip: ' + "%.2f"%mortgage.interest_payment[step] +
         ' | total pp: ' + "%.2f"%paid_principal +
         ' | total ip: ' + "%.2f"%paid_interest)

In [ ]:
inflation = 0.03
market_trend = 0.02

for step in range(0,10):
    flat.projectPrice(step, inflation, market_trend)
    print('[step ' + str(step) + '] flat price: ' + "%.2f"%flat.price[step])

In [ ]:
property_tax= 25
renovation_costs = 200
renovation_steps = {5, 10}
house_community_costs = 15
    
for step in range(0,11):
    flat.projectCosts(step, property_tax, renovation_costs, renovation_steps, house_community_costs)
    print('[step ' + str(step) + '] flat costs: ' + "%.2f"%flat.costs[step])